# Logistic Regression from scratch

**Author:** Umberto Michelucci, (C) 2017

In this review I do a complete discussion of logistic regression for binary classification. I perform all the mathematical derivations, including backpropagation and optimisation. I derive the necessary equations for one training cases, and then I derive the ones for $m$ training cases.

## Introduction

Let's suppose we want to be able to classify some observations in two classes. For example we may want to know if there is a cat in an image or not. Or we may want to know if a person will change insurance in the next six months or not.
Basically we will have some features that describe what we are observing (for example the RGB values of the pixel of the image, or the age, address, weight of a person) and we want to be able to predict if the observation is of class one or two (cat or no-cat, change insurance or not). This kind of classification is called binary classification.
Logistic regression, in its easier form, is used to perform exactly this. 

Our goal is to be able to have a model that we can train (let it learn from examples) and that can predict if a certain new observation (its input) is of one of two classes.

Let's first clarify some notation that we will use in this paper.

Our prediction will be a variable $a$ that can only be $0$ or $1$ (we will indicate with $0$ and $1$ the two classes we are trying to predict). In a more mathematical form we will have
$$
\text{prediction / estimate}\ \ \rightarrow \ \ a
\in \{0,1\}
$$

But what our method will give as output, or as a prediction, will be the probability of $a$ being 1, given the input case $x$. Or in a more mathematical form:

$$
a = P(y = 1 \ |\ x)
$$

usually we will then define an input observation to be of class $1$ if $a>0.5$ and of classe $0$ if $a <= 0.5$.

Let's assume we have $n_x$ input features (let's assume they are numerical): $x_1, x_2, ..., x_{n_x}$. We will indicate the vector with $x$ 

$$
x = (x_1, x_2, ..., x_{n_x})
$$

In our discussion we will also use the vector $w$ that will contain $n_x$ weigths (also numerical) and a constant $b$ (number) (called bias):

$$
w = (w_1, w_2, ..., w_{n_x})
$$


As you may know, what we need to do is to find the ideal weights $w$ and bias $b$ to classify our observations in the best way possible (what exactly means will be discussed later)

Let's suppose for a moment that we have already found the ideal $w$ and $b$.
To apply the method we will then have to perform the following steps

### Step 1

We first build a linear combination $z$ of our inputs using $w$ and $b$

$$
z = w_1 x_1+w_2 x_2+...+w_{n_x} x_{n_x}+b
\tag{1}
$$

Now it will be very useful to consider our vectors $x$ or $w$ as tensors (or matrices) of dimensions $(n_x, 1)$. This will make the generalisation to many training cases a lot easier, since we will be able to use the formula we will find (and that we will need to vectorize, more on that later) with minor modifications. So we have $x$ and $w$ and both have the dimensions $(n_x,1)$. Equation (1) can then be rewritten as a matrix multiplication.

$$
z = w^T x + b = w_1 x_1+w_2 x_2+...+w_{n_x} x_{n_x}+b
$$

### Step 2

We will then use the sigmoid function applied to $z$

$$
a = \sigma (z) = \sigma (w^T x + b)
$$

where with $w^T$ we have indicated the transpose of the vector $w$ (as already explained above). Note that this can have values only between 0 and 1, so it behaves as a probability.

Now we have assumed to have already found the ideal weights $w$ and bias $b$ so that the $a$ we get from the previous equation is already our prediction. We don't need to do anything else. But the whole idea of Machine Learning is, well, learning. So that means that we need to find the ideal $w$ and $b$ given a set of features (the $x$ in our notation).
But how to learn? 

## Cost Function

To find the best parameters $w$ and $b$, we will need to minimize what is called the cost function (CF). A complete discussion about it is outside the scope of this paper. The CF we will use is the cross entropy (to be precise the following equation is for the Loss function, but as will be clear later on, for one observation the Loss and the Cost function are the same).

$$
\mathscr{L} (a,y) = -\left[ y \log a + (1-y) \log (1-a) \right]
$$

It is useful to draw a computational graph to show the steps necessary to calculate $\mathscr{L} (a,y)$.
Please note that we are still discussing a single training observation. We will generalize the approach in the next section.

So to reiterate the idea we will need to find the parameters $w$ and $b$ that minimize $\mathscr{L} (a,y)$.

The computational graph to calculate $\mathscr{L} (a,y)$ is the following

![title](logistic_regression_computational_graph.png)

To minimize $\mathscr{L} (a,y)$ we will use the gradient descent algorithm. For a theoretical discussion of the method (well worth knowing) you can refer to several books (or to get an idea from Wikipedia: https://en.wikipedia.org/wiki/Gradient_descent)

So to be able to use the algorithm we will need to have all the partial derivatives of $\mathscr{L} (a,y)$ with respect to the weights $w_j$ (with $j \in {1,..., n_x})$ and $b$. So we will need to calculate

$$\displaystyle
\frac{\partial \mathscr{L} (a,y)}{\partial w_j}
\tag{2}
$$
and
$$\displaystyle
\frac{\partial \mathscr{L} (a,y)}{\partial b}
\tag{3}
$$

then during the "descent", at step $n+1$ we will need to update the parameters from step $n$. 
Let's indicate the parameter $w_j$ at step $n$ with the notation $w_{j,[n]}$ and $b$ with $b_{[n]}$.
We can then use the formulas

$$\displaystyle
w_{j, [n+1]} = w_{j,[n]} -\alpha \frac{\partial \mathscr{L} (a,y)}{\partial w_j} \ \ \ \ j \in \{1,2, ..., n_x\}
\tag{3a}
$$
and
$$\displaystyle
b_{[n+1]} = b_{[n]} -\alpha \frac{\partial \mathscr{L} (a,y)}{\partial b}
\tag{3b}
$$

where $\alpha$ is called the learning rate and is an additional parameter that needs to be optimized. But for the moment we will consider it as a constant.

So if we want to implement the entire algorithm from scratch we will need explicit equations for the partial derivatives. 

To calculate the derivatives in equation (2) and (3) we can simply apply the chain rule (for those who knows calculus). $\mathscr{L} (a,y)$ is simply the composition of several function. So we can easily write (try to derive it yourself)

$$\displaystyle
\frac{\partial \mathscr{L} (a,y)}{\partial w_j} = \frac{\partial \mathscr{L} (a,y)}{\partial a} \frac{da}{dz}\frac{\partial z}{\partial w_j} \ \ \ \ j \in \{1,2, ..., n_x\}
\tag{4}
$$

and

$$\displaystyle
\frac{\partial \mathscr{L} (a,y)}{\partial b} = \frac{\partial \mathscr{L} (a,y)}{\partial a} \frac{da}{dz}\frac{\partial z}{\partial b}
\tag{5}
$$

Now we can easily calculate all the derivates appearing in (4) and (5)

$$\displaystyle
\frac{\partial \mathscr{L} (a,y)}{\partial a} = - \frac{y}{a}+ \frac{1-y}{1-a}
$$

and

$$\displaystyle
\frac{da}{dz} = a(1-a)
$$

and finally

$$\displaystyle
\frac{\partial z}{\partial w_j} = x_j
$$

$$\displaystyle
\frac{\partial z}{\partial b} = 1
$$


When we put together those equations with (4) and (5) we get

$$\displaystyle
\frac{\partial \mathscr{L} (a,y)}{\partial w_j} = \frac{\partial \mathscr{L} (a,y)}{\partial a} \frac{da}{dz}\frac{\partial z}{\partial w_j} = (a-y) \ x_j
\tag{6}
$$

and

$$\displaystyle
\frac{\partial \mathscr{L} (a,y)}{\partial b} = \frac{\partial \mathscr{L} (a,y)}{\partial a} \frac{da}{dz}\frac{\partial z}{\partial b} = (a-y)
\tag{7}
$$

So that (3a) and (3b) can be written easily as

$$\displaystyle
w_{j, [n+1]} = w_{j,[n]} -\alpha \frac{\partial \mathscr{L} (a,y)}{\partial w_j} = w_{j,n} - \alpha \ (a-y) \ x_j
\tag{8}
$$
and
$$\displaystyle
b_{[n+1]} = b_{[n]} -\alpha \frac{\partial \mathscr{L} (a,y)}{\partial b} = b_n - \alpha \ (a-y)
\tag{9}
$$

Equation (8) and (9) will be the ones we will use in our gradient descent implementation. If you have heard about backpropagation and computational graphs the following graph will get you immediately equations (5) and (6) (at least the first part, to obtain the $(a-y)x_j$ and $(a-y)$ you still need to perform some derivatives). Remember that in backpropagation you go from right to left, instead that from left to right. Each unit is divided in two halfs. The right one is used in the forward propagation and the left one in the backpropagation. This method is widely used in neural networks, and especially in deep learning. Since logistic regression can be obtained by a neural network with a single sigmoid unit we can use this method to get the right equations for the gradient descent.

![Computational Graph for the backpropagation algorithm](logistic_regression_computational_graph2.png)

## Generalisation to many training cases

Now we need to generalise our equation to the case when we have many training cases. This will be easier than expected. First some notation. Let's indicate with $x^{(i)}$ our $i$th training case. Remember $x^{(i)}$ have the dimension $(n_x,1)$. So they are basically "vertical" vectors. Or better formulated $x^{(i)}$ is a $\mathbb{R}^{n_x \times 1}$ matrix. Let's indicate with $X$ our matrix containing all the features and all the training cases. So $X$ will be a matrix of dimensions $\mathbb{R}^{n_x \times m}$ where with $m$ we have indicated the number of training cases that we have. $X$ can be obtained stacking vertically all the $x^{(i)}$.

So let's rewrite our equations in matrix form:

$$
Z = w^T X + B = (
\begin{matrix}
w^T x^{(1)}+b & w^T x^{(2)}+b & ... & w^T x^{(m)}+b  \\
\end{matrix}
)
$$

where we have indicated with $B$ a matrix of dimension $(1,n_x)$ with all elements equal to $b$. In Python we will not need to define this additional vector since **broadcasting** will take care of it.
Note that $Z$ will have the dimensions $(1, m)$, since $w^T$ has dimensions $(1,n_x)$ and $X$ has $(n_x,m)$.

Then we will need to calculate $A$. This will be a matrix of dimensions $(1,m)$ that will be

$$
A = \sigma (Z) = [
\begin{matrix}
\sigma (w^T x^{(1)}+b) &\sigma (w^T x^{(2)}+b) & ... & \sigma (w^T x^{(m)}+b)  \\
\end{matrix}
]
$$

### Cost Function

I mentioned before that the Loss function and the cost function for one training case are the same. When we are dealing with many training cases we will define our cost function as the average of our loss function calculated over all training cases as:

$$\displaystyle
J(w,b) = \frac{1}{m} \sum_{i=1}^{m} \mathscr{L} (a^{(i)},y^{(i)})
$$

Now we will need to calculate the derivative of our new cost function to implement the backpropagation algorithm to determine our parameters. This will not be difficult, since the derivative of the sum of functions is the sum of derivatives so we have simply

$$\displaystyle
\frac{\partial J(w,b)}{\partial w_j} = \frac{1}{m} \sum_{i=1}^{m} 
\frac{\partial J(w,b)}{\partial a^{(i)}} \frac{da^{(i)}}{dz^{(i)}}\frac{\partial z^{(i)}}{\partial w_j} = \frac{1}{m} \sum_{i=1}^{m}(a^{(i)}-y^{(i)}) \ x^{(i)}_j
\tag{8}
$$

and for $b$

$$\displaystyle
\frac{\partial J(w,b)}{\partial b} = \frac{1}{m} \sum_{i=1}^{m} 
\frac{\partial J(w,b)}{\partial a^{(i)}} \frac{da^{(i)}}{dz^{(i)}}\frac{\partial z^{(i)}}{\partial b} = \frac{1}{m} \sum_{i=1}^{m}(a^{(i)}-y^{(i)}) 
\tag{9}
$$

Now we can define the following matrix
$$\displaystyle
A = \left[
\begin{matrix}
a^{(1)}& a^{(2)} & ... & a^{(m)}  \\
\end{matrix}
\right] \ \ \ \in \mathbb{R}^{1 \times m}
$$

and we will have

$$
A-Y = \left[
\begin{matrix}
a^{(1)}-y^{(1)}& a^{(2)}-y^{(2)} & ... & a^{(m)}-y^{(m)}  \\
\end{matrix}
\right] \ \ \ \in \mathbb{R}^{1 \times m}
$$

We first will need to evaluate another matrix (since we have many parameters). So we will need to find (remember that w is not a scalar, and that has dimensions $(n_x,1)$)

$$
\nabla_w J(w,b) = \frac{1}{m} X^T (A-Y)
\tag{10}
$$

$$
\frac{\partial J(w,b)}{\partial b} = \frac{1}{m} \sum_{i=1}^{m}(A_i-Y_i) 
\tag{11}
$$

So with the equation (10) and (11) you can update the parameters to implement the gradient descent algorithm. So we can write the value of $w$ at step $n+1$ as

$$
w_{[n+1]} = w_{[n]}-\alpha \frac{1}{m} X^T (A-Y)
$$

and for $b$

$$
b_{[n+1]} = b_{[n]}-\alpha \frac{1}{m} \sum_{i=1}^{m}(A_i-Y_i) 
$$

This can be easily implemented in Python giving a general method to evaluate and find the optimal parameters.